# OpenSearch 벡터 스토어 및 Amazon Bedrock를 사용한 RAG 
> 이 노트북은  SageMaker Studio* **`Data Science 3.0`** kernel 및 ml.t3.medium 인스턴스에서 테스트 되었습니다.
---
### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

### 선수조건
- 임베딩 모델의 세이지 메이커 엔드포인트가 액티브 된 상태를 가정 합니다.
    - 세이지 메이커 엔드포인트에 배포하기 위해서는 아래 노트북을 실행하시고, Endpoint Name 만을 복사 하시면 됩니다.
    - [KoSIMCSERoberta Embedding Model 배포](https://github.com/gonsoomoon-ml/Kor-LLM-On-SageMaker/blob/main/1-Lab01-Deploy-LLM/4.Kor-Embedding-Model.ipynb)
    - SageMaker Endpoint 에 대해서는 공식 개발자 문서를 참조하세요 --> [Create your endpoint and deploy your model](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html)
- 오픈 서치 서비스가 액티브 된 상태를 가정 합니다.
---

OpenSearch는 대규모 데이터셋에 대한 유사도 검색을 위한 강력한 엔진입니다. Amazon OpenSearch Service를 통해 쉽게 클라우드 환경에서도 이용할 수 있습니다. 이와 함께 Vector Store를 사용하면 고차원 벡터 데이터를 효율적으로 저장하고 빠르게 검색할 수 있어, 복잡한 자연어 처리 작업을 더욱 간편하게 수행할 수 있습니다.


In [ ]:
import boto3
region = boto3.Session().region_name
opensearch = boto3.client('opensearch', region)

%store -r opensearch_user_id opensearch_user_password domain_name opensearch_domain_endpoint

try:
    opensearch_user_id
    opensearch_user_password
    domain_name
    opensearch_domain_endpoint
   
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Run 00_setup notebook first or Create Your Own OpenSearch Domain")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

# 1. Bedrock Client 생성

In [ ]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print(colored("\n== FM lists ==", "green"))
pprint(bedrock_info.get_list_fm_models(verbose=True))

# 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

## LLM 로딩 (Claude-v2)

In [ ]:
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [ ]:
# - create the Anthropic Model
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 512
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)
llm_text

## Embedding 모델 선택

In [ ]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):
    if is_bedrock_embeddings:

        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings

        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Embeddings-G1"
            )
        )

        print("Bedrock Embeddings Model Loaded")
        
    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )        
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")
    
    return llm_emb

#### [중요] is_KoSimCSERobert == True 일시에 endpoint_name 을 꼭 넣어 주세요.

In [ ]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)     

# 3. 데이터 준비


##  신한은행 FAQ 데이터 세트로 구현
- [중요] 저자 및 동료가 아래의 웹사이트에서 크로링한 기준으로 구성 하였습니다.
- 인터넷뱅킹 FAQ > 스마트뱅킹 No.1 ~ N. 89 로 구성되었습니다. 
- https://www.shinhan.com/hpe/index.jsp#050101020000

In [ ]:
import pandas as pd
pd.options.display.max_rows = 20

data_file_path = "data/fsi_smart_faq_ko.csv"
df = pd.read_csv(data_file_path)
df

## 데이터 전처리
- 여기서 no 는 제거 합니다. 

In [ ]:
def preprocess_data(df):
    ldf = df.copy()

    ldf.rename(columns={'Category': 'ask'}, inplace=True)
    df_index = ldf.drop(['no'], axis=1)
    # df_index.to_csv("rag_data_kr/fsi_smart_faq_ko_answer.csv", index=None, header=None)
    df_index.to_csv("data/fsi_smart_faq_ko_preprocess.csv", index=None)

    return df_index

pre_df = preprocess_data(df)
pre_df.head(3)

### CSVLoader 로 문서 로딩

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

In [ ]:
loader = CSVLoader(
    file_path="data/fsi_smart_faq_ko_preprocess.csv",
    # csv_args={
    #     "delimiter": ",",
    #     "fieldnames": ["Category", "Information", "type", "Source"],
    # },    
    source_column="Source",
    encoding="utf-8"
)


## 메타 데이터 생성
- 기존 Category 컬럼을 ask 로 변경 합니다.
- 컬럼의 type, source 는 metadata 로 생성하고, 내용에서는 삭제 합니다.
- 타임스탬프 및 임베딩 모델의 엔드포인트 이름을 metadata 로 추가 합니다.

In [ ]:
import time
documents_fsi = loader.load()

def create_metadata(docs):
    # # add a custom metadata field, such as timestamp
    for idx, doc in enumerate(docs):
        # type 을 메타 데이타로 저장
        stype = doc.page_content.split("type: ")[1].split("\n")[0]
        # print("stype: ", stype)
        split_content = doc.page_content.split("type: ")
        content = split_content[0]        
        metadata = split_content[1]                
        doc.metadata['type'] = metadata.split("\n")[0]        
        doc.page_content = content # metadata 제외하고 content 만 저장
        # doc.metadata['type'] = doc.page_content.split("type: ")[1].split("\n")[0]
        doc.metadata['timestamp'] = time.time()

        
create_metadata(documents_fsi)


In [ ]:
print(len(documents_fsi))

documents_fsi[0]
# documents_fsi[-1]

## Text Spliter 로 청킹
참고: 검색된 문서/텍스트는 질문에 대답하기에 충분한 정보를 포함할 만큼 커야 합니다. 하지만 LLM 프롬프트에 들어갈 만큼 충분히 작습니다. 또한 임베딩 모델에는 입력 토큰 길이가 512개 토큰으로 제한되어 있으며 이는 한국어의 경우에 대략 180 ~ 200 글자로 변환됩니다. 이 사용 사례를 위해 [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html)를 사용하여 210자가 겹치는 약 50자의 청크를 생성합니다.

In [ ]:
if is_bedrock_embeddings:
    chunk_size = 2048
    chunk_overlap = 0
elif is_KoSimCSERobert:
    chunk_size = 800 # This is maxumum
    chunk_overlap = 0
    

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    separators=["\n\n", "\n", ".", " ", ""],
    length_function = len,
)

docs = text_splitter.split_documents(documents_fsi)
print(f"Number of documents after split and chunking={len(docs)}")

## 청킹 통계 및 내용 확인

In [ ]:
from utils.rag import show_context_used, show_chunk_stat
    
show_chunk_stat(docs)

In [ ]:
show_context_used(docs, limit=5)

# 4. OpenSearch 벡터 Indexer 생성
### 선수 조건
- 아래의 링크를 참조해서 OpenSearch Service 를 생성하고, opensearch_domain_endpoint, http_auth 를 복사해서, 아래 셀의 내용을 대체 하세요.
    - [OpenSearch 생성 가이드](https://github.com/gonsoomoon-ml/Kor-LLM-On-SageMaker/blob/main/2-Lab02-QA-with-RAG/4.rag-fsi-data-workshop/TASK-4_OpenSearch_Creation_and_Vector_Insertion.ipynb)
- 랭체인 오프서처 참고 자료
    - [Langchain Opensearch](https://python.langchain.com/docs/integrations/vectorstores/opensearch)

#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.
- 20_applications/02_qa_chatbot/01_preprocess_docs/01_parameter_store_example.ipynb 참고

In [ ]:
http_auth = (opensearch_user_id, opensearch_user_password) # Master username, Master password
index_name = "genai-demo-index-v1"

## 오픈 서치 인덱스 유무에 따라 삭제
오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [ ]:
from utils.opensearch import opensearch_utils

In [ ]:
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

index_exists = opensearch_utils.check_if_index_exists(
    os_client,
    index_name
)

if index_exists:
    opensearch_utils.delete_index(
        os_client,
        index_name
    )
else:
    print("Index does not exist")

In [ ]:
from langchain.vectorstores import OpenSearchVectorSearch

In [ ]:
%%time
# by default langchain would create a k-NN index and the embeddings would be ingested as a k-NN vector type
docsearch = OpenSearchVectorSearch.from_documents(
    index_name=index_name,
    documents=docs,
    embedding=llm_emb,
    opensearch_url=opensearch_domain_endpoint,
    http_auth=http_auth,
    bulk_size=10000,
    timeout=60
)

## 인덱싱된 벡터 확인

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

In [ ]:
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2"
)

## OpenSearch Dashboard 통한 Query 및 레코드 확인
- OpenSearch Dashboards URL 은 Amazon OpenSearch Servie 콘솔에 가시면 확인할 수 있습니다.
OpenSearch DevTools Console DSL 방법
- [OpenSearch_Query_DSL](https://opensearch.org/docs/latest/query-dsl/index/)

![record_opensearch](img/record_opensearch.jpg)

# 5.오픈 서치에 "유사 서치" 검색
- query 를 제공해서 실제로 유사한 내용이 검색이 되는지를 확인 합니다.



- similarity_search_with_score API 정보
    - [API: similarity_search_with_score](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html#langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.similarity_search)

In [ ]:
from langchain.chains.question_answering import load_qa_chain

In [ ]:
def get_semantic_similar_docs(**kwargs):

    search_types = ["approximate_search", "script_scoring", "painless_scripting"]
    space_types = ["l2", "l1", "linf", "cosinesimil", "innerproduct", "hammingbit"]

    assert "vector_db" in kwargs, "Check your vector_db"
    assert "query" in kwargs, "Check your query"
    assert kwargs.get("search_type", "approximate_search") in search_types, f'Check your search_type: {search_types}'
    assert kwargs.get("space_type", "l2") in space_types, f'Check your space_type: {space_types}'

    results = kwargs["vector_db"].similarity_search_with_score(
            query=kwargs["query"],
            k=kwargs.get("k", 5),
            search_type=kwargs.get("search_type", "approximate_search"),
            space_type=kwargs.get("space_type", "l2"),
            #boolean_filter=kwargs.get("boolean_filter", {}),        
            boolean_filter=opensearch_utils.get_filter(
                filter=kwargs.get("boolean_filter", [])
            ),        
            # fetch_k=3,
        )

    print ("\nsemantic search args: ")
    pprint ({
        "k": kwargs.get("k", 5),
        "search_type": kwargs.get("search_type", "approximate_search"),
        "space_type": kwargs.get("space_type", "l2"),
        "boolean_filter": opensearch_utils.get_filter(filter=kwargs.get("boolean_filter", []))
    })
    
    if kwargs.get("hybrid", False) and results:            
        max_score = results[0][1]
        new_results = []
        for doc in results:
            nomalized_score = float(doc[1]/max_score)
            new_results.append((doc[0], nomalized_score))
        results = copy.deepcopy(new_results)

    return results

## Filter 없이 실행. 

In [ ]:
query = '간편조회서비스는 회원가입해야하나요?'
#query = "타기관OTP 등록 방법 알려주세요"

pre_similar_doc = get_semantic_similar_docs(
    vector_db=vector_db,
    query=query,
    k=3
)
opensearch_utils.opensearch_pretty_print_documents(pre_similar_doc)

## metadata의 type, source 에 필터를 걸어서 검색

In [ ]:
filter01 = "인터넷뱅킹"
#filter01 = "인증서"
filter02 = "신한은행"
#filter02 = "아마존은행"

boolean_filter = [
    {"term": {"metadata.type": filter01}},
    {"term": {"metadata.source": filter02}},
]

pre_similar_doc = get_semantic_similar_docs(
    vector_db=vector_db,
    query=query,
    boolean_filter=boolean_filter,
    k=3
)
opensearch_utils.opensearch_pretty_print_documents(pre_similar_doc)

# 6.사용자 정의 가능한 옵션
이제 벡터 저장소가 준비되었으므로 질문을 시작할 수 있습니다.

Vector Store를 둘러싸서 LLM 입력을 받는 LangChain에서 제공하는 래퍼를 사용할 수 있습니다.
이 래퍼는 뒤에서 다음 단계를 수행합니다.
- 질문을 입력합니다.
- 질문 임베딩 생성
- 관련 문서 가져오기
- 프롬프트에 문서와 질문을 채워 넣습니다.
- 프롬프트로 모델을 호출하고 사람이 읽을 수 있는 방식으로 답변을 생성합니다.

위 시나리오에서는 질문에 대한 상황 인식 답변을 빠르고 쉽게 얻을 수 있는 방법을 탐색했습니다. 이제 문서를 가져오는 방법을 사용자 정의할 수 있는 [RetrievalQA](https://python.langchain.com/en/latest/modules/chains/index_examples/Vector_db_qa.html)의 도움으로 더 사용자 정의 가능한 옵션을 살펴보겠습니다. `chain_type` 매개변수를 사용하여 프롬프트에 추가해야 합니다. 또한 검색해야 하는 관련 문서 수를 제어하려면 아래 셀에서 'k' 매개변수를 변경하여 다른 출력을 확인하세요. 많은 시나리오에서 LLM이 답변을 생성하는 데 사용한 소스 문서가 무엇인지 알고 싶을 수 있습니다. LLM 프롬프트의 컨텍스트에 추가된 문서를 반환하는 `return_source_documents`를 사용하여 출력에서 ​​해당 문서를 가져올 수 있습니다. 'RetrievalQA'를 사용하면 모델에 특정한 사용자 정의 [프롬프트 템플릿](https://python.langchain.com/en/latest/modules/prompts/prompt_templates/getting_started.html)을 제공할 수도 있습니다.

참고: 이 예에서는 Amazon Bedrock에서 LLM으로 Anthropic Claude를 사용하고 있습니다. 이 특정 모델은 입력이 'Human:' 아래에 제공되고 모델이 'Assistant:' 다음에 출력을 생성하도록 요청되는 경우 가장 잘 수행됩니다. 아래 셀에는 LLM이 기본 상태를 유지하고 컨텍스트 외부에서 응답하지 않도록 프롬프트를 제어하는 ​​방법의 예가 나와 있습니다.

### [TIP] Prompt의 instruction의 경우 한글보다 **영어**로 했을 때 더 좋은 결과를 얻을 수 있습니다.

### [[REF] Using langchain for Question Answering on Own Data](https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [ ]:
# prompt_template = """
# \n\nHuman: 다음 문맥의 Information을 사용하여 고객 서비스 센터 직원처럼, 마지막 질문에 대한 목차 형식으로 답변을 제공하세요. 응답을 모르면 모른다고 말하고 응답을 만들려고 하지 마세요.

# {context}

# Question: {question}
# \n\nAssistant:"""

prompt_template = """
\n\nHuman: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}

\n\nAssistant:"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

def run_RetrievalQA(**kwargs):

    chain_types = ["stuff", "map_reduce", "refine"]

    assert "llm" in kwargs, "Check your llm"
    assert "query" in kwargs, "Check your query"
    assert "prompt" in kwargs, "Check your prompt"
    assert "vector_db" in kwargs, "Check your vector_db"
    assert kwargs.get("chain_type", "stuff") in chain_types, f'Check your chain_type, {chain_types}'

    qa = RetrievalQA.from_chain_type(
        llm=kwargs["llm"],
        chain_type=kwargs.get("chain_type", "stuff"),
        retriever=kwargs["vector_db"].as_retriever(
            search_type="similarity",
            search_kwargs={
                "k": kwargs.get("k", 5),
                "boolean_filter": opensearch_utils.get_filter(
                    filter=kwargs.get("boolean_filter", [])
                ),
            }
        ),
        return_source_documents=True,
        chain_type_kwargs={
            "prompt": kwargs["prompt"],
            "verbose": kwargs.get("verbose", False),
        },
        verbose=kwargs.get("verbose", False)
    )

    return qa(kwargs["query"])

In [ ]:
from utils.rag import run_RetrievalQA

In [ ]:
#query = '간편조회서비스는 회원가입해야하나요?'
#query = "타기관OTP 등록 방법 알려주세요"
query = "홈페이지 탈퇴하는방법 알려줘"

result = run_RetrievalQA(
    query=query,
    llm=llm_text,
    prompt=PROMPT,
    vector_db=vector_db,
    verbose=False,
    k=3
)

print("##################################")
print("query: ", query)
print("##################################")
print_ww(result['result'])

In [ ]:
show_context_used(result['source_documents'])

In [ ]:
query = "홈페이지 탈퇴하는방법 알려줘"

filter01 = "홈페이지"
# filter01 = "인증서"
filter02 = "신한은행"
# filter02 = "아마존은행"

boolean_filter = [
    {"term": {"metadata.type": filter01}},
    {"term": {"metadata.source": filter02}},
]

result = run_RetrievalQA(
    query=query,
    boolean_filter=boolean_filter,
    llm=llm_text,
    prompt=PROMPT,
    vector_db=vector_db,
    verbose=False,
    k=3
)

print("##################################")
print("query: ", query)
print("boolean_filter: ", boolean_filter)
print("##################################")
print_ww(result['result'])

In [ ]:
show_context_used(result['source_documents'])

In [ ]:
query = "홈페이지 이용자아이디 여러 개 사용할 수 있나요?"
filter01 = "홈페이지"
# filter01 = "인증서"
filter02 = "신한은행"
#filter02 = "아마존은행"

boolean_filter = [
    {"term": {"metadata.type": filter01}},
    {"term": {"metadata.source": filter02}},
]

result = run_RetrievalQA(
    query=query,
    boolean_filter=boolean_filter,
    llm=llm_text,
    prompt=PROMPT,
    vector_db=vector_db,
    verbose=True,
    k=3
)

print("##################################")
print("query: ", query)
print("boolean_filter: ", boolean_filter)
print("##################################")
print_ww(result['result'])

In [ ]:
show_context_used(result['source_documents'])